# Сервис: Чат-бот по анализу токсичных сообщений

# 🎓 Шаг #1. Какая главные цели модели?! Какие ограничения?

- Максимальная задержка ответа (меньше 0.01 секундны на один запрос)
- Максимальный размер модели (до 1 GB)

# 📈 Шаг #2. Какая минимальная ML метрика?
- Пусть для простоты определяющей ML метрикой будет `Precision` = 97% 
- Помните, что для реальных бизнес целей иногда важнее определиться с метрикой `Recall` или `Precision`, а не `Accuracy` (Не самая лучшая метрика). 

(Выбор между двумя последними завист от того, какая ошибка для нас важнее  - пропустить токсичное сообщение, или посчитать нормальное не токсичным)

# 🚚 Шаг #3. Выбор модели - Языковая модель BERT.

<img src='https://images.squarespace-cdn.com/content/v1/56e2e0c520c6472a2586add2/1593683608007-L71NCKC2O54GFBHPB0W9/CP%2BLogos%2B2%2B%25288%2529.jpg' wide = 50>


* Страна, где живут многие модели: https://huggingface.co

* Нужная нам языковая модель: https://huggingface.co/SkolkovoInstitute/russian_toxicity_classifier?

### Запустим модель с huggingface

In [3]:
from transformers import BertTokenizer, BertForSequenceClassification

# Загружаем tokenizer и веса модели
tokenizer = BertTokenizer.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')
model = BertForSequenceClassification.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')

### Модель есть, протестируем её теперь на реальных примерах

In [6]:
%time # Засекаем время работы модели

text = 'Вау, ты просто супер!'

batch = tokenizer.encode(text, return_tensors='pt')

# inference
response = model(batch).logits.detach().numpy()

print(response)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs
[[ 0.26881275 -0.6409307 ]]


###  Оборачиваем модель в функцию

In [7]:
import numpy as np

def get_toxic_score(text):
    
    '''
    Вспомогательная функция, которая по тексту определяет уровень его токсичности
    '''
    
    batch = tokenizer.encode(text, return_tensors='pt')  # Используем модель
    response = model(batch).logits.detach().numpy()  # Получаем прогноз модели
    score = np.exp(response[0][1])/sum(np.exp(response[0])) # Оборачиваем в softmax для получения вероятности
    
    score = np.round(score, 3)
    
    if score > 0.6:
        return 'токсичное сообщение', score
    elif 0.10 >= score >= 0.60:
        return 'сообщение средней токсичности', score
    else:
        return 'сообщение не токсичное', score
    
get_toxic_score(text)

('сообщение не токсичное', 0.287)

In [8]:
%time # Замеряем время

text = 'Вау, ты просто супер!'  # Позитивный пример текста
get_toxic_score(text)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


('сообщение не токсичное', 0.287)

In [9]:
text = 'Это самое ужасное сообщение, ты худший автор !!!!! Ненавижу тебя!'  # Негативный пример текста
get_toxic_score(text)

('токсичное сообщение', 0.941)

In [10]:
text = 'Это канал Data Feeling! Автор канал Алерон Миленькин. Вот ссылка @datafeeling'  
get_toxic_score(text)

('сообщение не токсичное', 0.009)

# ✈️ Шаг #4. Создание сервиса (API). Обертка в Telegram чат-бота

Ссылка https://pypi.org/project/pyTelegramBotAPI/#a-simple-echo-bot

In [11]:
!pip install pyTelegramBotAPI -q

In [12]:
import telebot

from config import token

In [13]:
token= token
bot=telebot.TeleBot(token)

@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id, 'Привет')

    
@bot.message_handler(content_types='text')
def message_reply(message):
        
    text, score = get_toxic_score(message.text)
    
    answer = f'{text} \nПроцент токсичности: {score}'
    
    bot.send_message(message.chat.id, answer)
    
bot.infinity_polling() # запускаем бота, чтоб он работал вечно

2022-12-19 09:24:38,233 (__init__.py:970 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2022-12-19 09:24:38,233 (__init__.py:972 MainThread) ERROR - TeleBot: "Break infinity polling"


# 🔥 Шаг #5. Деплой на сервис 
- Переносим свой проект на GitHub (https://github.com/a-milenkin/deep_learn_school_deploy) 
- Обязательно указываем нужные библиотеки в `requirements.txt`
- Замечание: в данном примере для простты, установим библиотеки без использования виртуального кружения, но лучше так не делать



# ⚙️ Деплой на сервис PythonAnywhere.com


<img src='https://www.pythonanywhere.com/static/anywhere/images/PA-logo.svg' wide = 30>


Ссылка: https://www.pythonanywhere.com/

* Клонируем проект на сервер `git clone https://github.com/a-milenkin/deep_learn_school_deploy.git`
* Входим в проект `cd deep_learn_school_deploy`
* Устанавливаем нужные библиотеки `pip install requirements.txt`
* Запускаем бота `python app.py`

# ⚙️ Альтернативный деплой на сервис Railway.app 

<img src='https://railway.app/brand/logotype-dark.png' wide = 30>

Ссылка https://railway.app/dashboard


* Добавялем в проект файл без расширения с названием `Procfile` с одной строчкой `web: python app.py`
* Авторизуемся через GitHub и деплоим проект. Все!
* Возьмем еще более легкую модель

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = 'cointegrated/rubert-tiny-toxicity'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
    
def get_toxic_score(text):
    """ Calculate toxicity of a text (if aggregate=True) or a vector of toxicity aspects (if aggregate=False)"""
    
    batch = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
    response = model(**batch).logits.detach().numpy()
    score = np.exp(response[0][0])/sum(np.exp(response[0]))
    
    score = np.round(1 - score, 3)
    
    if score > 0.6:
        return 'токсичное сообщение', score
    elif 0.10 >= score >= 0.60:
        return 'сообщение средней токсичности', score
    else:
        return 'сообщение не токсичное', score

print(get_toxic_score('я люблю тебя'))

# Дополнительный материал

* Ссылкан готовый код на GitHub https://github.com/a-milenkin/deep_learn_school_deploy
* Документация по **Pythonanywhere.com** и **Railway.app**
* Более подобный тьюториал по Pythonanywhere - https://pythonhelp.ru/post/2018-11-24-pythonanywhere/
* Гайди по Railway.app https://docs.railway.app/